In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import os
import sys
import itertools
import numpy as np
from numpy import array, arange, abs as np_abs
from scipy import signal
from numpy.fft import rfft, rfftfreq
import math
from sklearn.cluster import KMeans
import glob
from sklearn import preprocessing
from scipy import interpolate
import scipy
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans

In [3]:
def signalnoiseratio(x,y):    
    
    signalPow = math.sqrt(sum(x*np.conjugate(x)))
    noisePow = math.sqrt(sum(y*np.conjugate(y)))
    snr = 10 * np.log10(signalPow / noisePow)
    
    return snr

In [4]:
filename_ecg = glob.glob('ЭКГ/*.dat')
filename_class = glob.glob('Верификация/*cif.arr')
filename_marks = glob.glob('Границы/*.Marks')

In [5]:
def for_open_files(filename_ecg, filename_class, filename_marks, i, error):    
    
    f = open(filename_ecg[i], "r")
    Signal = np.fromfile(f, dtype=np.uint16)
    Signal.resize(int(len(Signal)/3),3)
    Class = np.genfromtxt (filename_class[i],delimiter='')
    Marks = np.genfromtxt (filename_marks[i],delimiter='')
    if error:
        Marks = len_qrs_bias(Marks, Class)
    else: 
        None
    
    return Signal, Class, Marks

In [6]:
def len_qrs_bias(Marks, Class):
    
    
    # t - коэффициент стьюдента
    L = len(Marks[:,1]-Marks[:,0])
    M = np.mean(Marks[:,1]-Marks[:,0])
    S = np.std(Marks[:,1]-Marks[:,0])
    new_min = M-3*S
    new_max = M+3*S
    len_new_marks = np.random.randint(new_min, new_max, size=len(Marks))
    bias = (L - len_new_marks)/2
    bias = np.array([int(i) for i in bias])
    Marks_new = np.copy(Marks)
        
    Marks_new[:,0] = Marks_new[:,0] - bias
    Marks_new[:,1] = Marks_new[:,1] + bias
    
    for i in range(len(Marks_new)):
        if Marks_new[i,1] < Class[i,2]:
            Marks_new[i,1] = Marks_new[i,1] + 2*bias[i]
        if Marks_new[i,0] > Class[i,2]:
            Marks_new[i,0] = Marks_new[i,0] - 2*bias[i]
    
    # чекнуть на пидора
    if Marks_new[0,0] <= 6:
        Marks_new[0,0] = Marks[0,1]
    if Marks_new[-1,1] >= 15000:
        Marks_new[-1,1] = Marks[-1,1]
    if Marks_new[0,0] > Class[0,2]:
        Marks_new[0,0] = Marks[0,1]
    if Marks_new[-1,1] < Class[-1,2]:
        Marks_new[-1,1] = Marks[-1,1]

    return Marks_new

In [7]:
def for_noisy_signal(Signal, ratio):
    
    # функция для зашумления исходного сигнала
    if ratio == 0:
        signal_after_noisy_1 = Signal[:,0]
        signal_after_noisy_2 = Signal[:,1]
        signal_after_noisy_3 = Signal[:,2]
        signal_after_noisy = np.column_stack((signal_after_noisy_1, signal_after_noisy_2, signal_after_noisy_3))
    else:
        signal_after_noisy_1 = noisy(Signal[:,0],ratio)[0]+Signal[:,0]
        signal_after_noisy_2 = noisy(Signal[:,1],ratio)[0]+Signal[:,1]
        signal_after_noisy_3 = noisy(Signal[:,2],ratio)[0]+Signal[:,2]
        signal_after_noisy = np.column_stack((signal_after_noisy_1, signal_after_noisy_2, signal_after_noisy_3))
    
    return signal_after_noisy

In [8]:
# создаем шум
def noisy(x,ratio):

    k = 1
    y = (np.random.normal(0, 1, len(x))) * k
    nyq = 0.5 * 250
    low = 5 / nyq
    high = 35 / nyq
    b, a = signal.butter(2, [low, high], btype='band')
    y = signal.filtfilt(b, a, y)
    snr = signalnoiseratio(x,y)
    while snr > ratio: 
        y = (np.random.normal(0, 1, len(x))) * k
        y = signal.filtfilt(b, a, y)
        snr = signalnoiseratio(x,y)
        k += 1
        
        #print(snr)

    return y, snr

In [9]:
def signalnoiseratio(x,y):    
    
    signalPow = math.sqrt(sum(x*np.conjugate(x)))
    noisePow = math.sqrt(sum(y*np.conjugate(y)))
    snr = 10 * np.log10(signalPow / noisePow)
    
    return snr

In [10]:
def preprocess(Signal):
    
    # Функция для предварительной обработки сигнала (фильтрация)
    
    nyq = 0.5 * 250
    low = 5 / nyq
    high = 20 / nyq
    b, a = signal.butter(2, [low, high], btype='band')    
    signal_after_processing_1 = signal.filtfilt(b, a, Signal[:,0])
    signal_after_processing_2 = signal.filtfilt(b, a, Signal[:,1])
    signal_after_processing_3 = signal.filtfilt(b, a, Signal[:,2])
    signal_after_processing = np.column_stack((signal_after_processing_1, signal_after_processing_2, signal_after_processing_3))
    
    return signal_after_processing

In [11]:
def informative_features(signal_after_processing, Marks, Class, pca, features):
    
    if len(features) == 1:
        pca = False
    
    Fs = 250
    inf_features_1 = []
    inf_features_2 = []
    inf_features_3 = []
    inf_features_4 = []
    for i in range(len(Marks)):
        # выделение QRS-комплекса
        QRS_complexes = signal_after_processing[int(Marks[i,0]):int(Marks[i,1])]
        for l in range(3):
            inf_features_one_lead = []
            QRS = QRS_complexes[:,l]
            # создание изолинии
            QRS_end = len(QRS)-1
            k = ((QRS[0] - QRS[QRS_end])/(1-len(QRS)))
            b = (QRS[0] - ((QRS[0]-(QRS[QRS_end]))/(1-len(QRS))*1))
            isoline = [(k*i + b) for i in range(len(QRS))]
            # для частотных признаков (фурьешечка)
            QRS_spectrum = np_abs(rfft(QRS, n=None, axis=-1))
            F = rfftfreq(len(QRS), d=1/Fs) 
            # тангенс угла наклона (скорость нарастания сигнала)
            if 1 in features:
                QRS_square = QRS*QRS 
                inf_features_one_lead.append(np.ediff1d(QRS_square)[int(Class[i,2]-Marks[i,0]-5)]) #1
            if 2 in features:
            # площадь QRS-комплекса 
                inf_features_one_lead.append(sum(abs(QRS-isoline))) #2
            # пик в частотном спектре
            if 3 in features:    
                inf_features_one_lead.append(F[int(np.where(QRS_spectrum == max(QRS_spectrum[(F > 5) & (F < 35)]))[0])]) #3
            # мощность QRS-комплекса в заданном частотном диапазоне
            if 4 in features:
                inf_features_one_lead.append(sum(QRS_spectrum[(F > 5) & (F < 35)])) #4
            # размах QRS-комплекса
            if 5 in features:   
                inf_features_one_lead.append(max(QRS)-min(QRS)) #5
            
            if l == 0:
                inf_features_1.append(inf_features_one_lead)
            elif l == 1:
                inf_features_2.append(inf_features_one_lead)
            elif l == 2:
                inf_features_3.append(inf_features_one_lead)
    
    if 6 in features:
        # длительность QRS-комплекса
        len_qrs = preprocessing.scale(Marks[:,1]-Marks[:,0]) #6
        inf_features_4 = len_qrs
    if 7 in features:
        RR_ratio = ratio_rr(Class) #отношения RR 
        try:
            inf_features_4 = np.column_stack((inf_features_4, RR_ratio))
        except:
            inf_features_4 = RR_ratio
    if pca:
        num_k = 2 # количество компонент
        pca_inf_features_1 = PCA(n_components=num_k).fit_transform(preprocessing.scale(inf_features_1))
        pca_inf_features_2 = PCA(n_components=num_k).fit_transform(preprocessing.scale(inf_features_2))
        pca_inf_features_3 = PCA(n_components=num_k).fit_transform(preprocessing.scale(inf_features_3))
        try:
            inf_features = np.column_stack((pca_inf_features_1,pca_inf_features_2,pca_inf_features_3, inf_features_4))  
        except:
            inf_features = np.column_stack((pca_inf_features_1,pca_inf_features_2,pca_inf_features_3))  
    else:
        try:
            inf_features = np.column_stack((inf_features_1,inf_features_2,inf_features_3, inf_features_4))  
        except:
            inf_features = np.column_stack((inf_features_1,inf_features_2,inf_features_3))

    return inf_features

In [12]:
def for_cluster_number_elbow(X):
    # переписать мб
    inertia = []
    for k in range(1, 10):
        kmeans = KMeans(n_clusters=k, random_state=1).fit(X)
        inertia.append(np.sqrt(kmeans.inertia_))
    
    clusters = []
    for i in range(len(inertia)-1):
        clusters.append(((inertia[i]-inertia[i+1])/inertia[i])*100)
    
    #find_k = [((clusters[i]-clusters[i+1])/clusters[i])*100 for i in range(len(clusters)-1)]
    #k = int(np.where(find_k == max(find_k))[0]+1)

    k = 1
    for i in range(len(clusters)):
        if ((clusters[i]-clusters[i+1])/clusters[i])*100 > 50:
            k += 1
        else:
            break
    
    return k

In [13]:
def for_cluster_number_silhouette(X):

    # переписать мб
    k = []
    for n_cluster in range(2, 11):

        kmeans = KMeans(n_clusters=n_cluster).fit(X)
        label = kmeans.labels_
        sil_coeff = silhouette_score(X, label, metric='euclidean') # мб на тест выбора метрики
        k.append(sil_coeff)
    k = int(np.where(k == max(k))[0] + 2)
    
    return k

In [14]:
def for_efficiency(Y,y_pred,k):

    # попробовать переписать через словари
    array_k = [i for i in range(k)]
    array_clusters = np.array([i+1 for i in range(k)])
    arrays = list(itertools.permutations(array_clusters))
    eff = []
    P_array = []
    for il in range(len(arrays)):
        z = np.zeros((k,len(y_pred)))
        array = arrays[il]
        for i in range(len(y_pred)):
            for l in range(k):
                if y_pred[i] == array_k[l]:
                    z[l,i] = 1
        array_test = np.dot(array,z)
        non_error = 0
        for i in range(len(array_test)):
            if Y[i] == array_test[i]:
                non_error += 1
        
        P_array.append(non_error)
        eff.append((non_error/len(array_test))*100)
        
    final_eff = max(eff)
    ind = eff.index(max(eff))
    final_result = np.dot(arrays[ind],z)
        
    acc = P_array[ind]/len(Y) * 100    
    
        
    return final_eff, acc, final_result, P_array[ind]

In [15]:
def ratio_rr(Class):

    RR_ratio = []
    RR = []
    RR.append(Class[1,2] - Class[0,2])
    for i in range(len(Class[:,2])-1):
        RR.append(Class[i+1,2] - Class[i,2])
    RR.append(Class[-1,2] - Class[-2,2]) 
    for i in range(len(RR)-1):
        RR_ratio.append(RR[i]/RR[i+1])
    RR_ratio = preprocessing.scale(RR_ratio)
    #RR = preprocessing.scale(RR_ratio[1:-2]) 
    return RR_ratio#, RR

In [16]:
column = ['features', 'pca', 'n_clust', 'method_clust', 'level_noise', 'error_duration','P', 'N', 'ACC', 'error_sg']
condition = pd.DataFrame(columns = column)
pre_condition = pd.DataFrame(columns = column)
final_condition = pd.DataFrame(columns = column)
[i for i in range(1,8)]
features = []
inf_f = [i for i in range(1,8)]
for i in range(1,8):
    features_for_append = list(itertools.combinations(inf_f, i))
    for l in range(len(features_for_append)):
        features.append(list(features_for_append[l]))
pca = [True, False]
n_clust = ['elbow','silhouette']
method_clust = ['KM', 'SC', 'AC']
level_noise = ['0','5','10']
error_duration = [False, True]
pre_condition['features'] = features
condition = condition.append(pre_condition)
array = [pca, n_clust, method_clust, level_noise, error_duration]
array_name = column[1:]
for l in range(len(array)):
    arr = array[l]
    condition = pre_condition
    for i in range(len(arr)):
        pre_condition[array_name[l]] = arr[i]
        condition = condition.append(pre_condition)
        #final_condition = final_condition.reset_index(drop=True)
        #final_condition = final_condition.append(condition)
    if i == len(arr)-1:
        pre_condition = condition
        #print(array_name[l])
condition = condition.reset_index(drop=True)

In [ ]:
for l in range(len(condition)):
    try:
        features = condition.features[l]
        pca = condition.pca[l]
        n_clust = condition.n_clust[l]
        method_clust = condition.method_clust[l]
        error = condition.error_duration[l]
        ratio = int(condition.level_noise[l])

        accuracy = []
        P = []
        N = []
        error_sg = 0
        for i in range(len(filename_ecg)):
            try:
                Signal, Class, Marks = for_open_files(filename_ecg, filename_class, filename_marks, i, error)
                signal_after_noisy = for_noisy_signal(Signal, ratio)
                signal_after_processing = preprocess(signal_after_noisy)

                inf_features = informative_features(signal_after_processing, Marks, Class, pca, features)
                #k = for_cluster_number_elbow(inf_features)
                k = for_cluster_number_silhouette(inf_features)
                y_pred = AgglomerativeClustering(n_clusters=k).fit_predict(inf_features)
                accuracy.append(for_efficiency(Class[:,3],y_pred,k)[1])
                P.append(for_efficiency(Class[:,3],y_pred,k)[3])
                N.append(len(Class[:,3]))
                print(i)
            except:
        #acc = for_efficiency(marks_qrs,y_pred,k)[1]
                print(filename_ecg[i],i)
                error_sg += 1
        condition.P[l] = sum(P)
        condition.N[l] = sum(N)
        condition.ACC[l] = round((sum(P)/sum(N))*100,2)
        condition.error_sg[l] = error_sg
        print(l,'rows')
    except:
        condition.error_sg[l] = -1
        print(l, 'rows error')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
0 rows
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
1 rows
0
1
2
3
4
5
6
7
ЭКГ\009.dat 8
ЭКГ\010.dat 9
10
11
ЭКГ\013.dat 12
13
14
ЭКГ\016.dat 15
ЭКГ\017.dat 16
ЭКГ\018.dat 17
ЭКГ\019.dat 18
19
20
21
22
23
24
25
26
27
28


In [ ]:
#признаки
'''

скорость нарастания - 1
площадь QRS-комплекса - 2
мощность - 3
частотак пика в частотном диапазоне - 4
размах - 5
длительность - 6
отношение RR-интервалов - 7

'''
#pca
'''
1 - True
2 - False
'''

# метод оптимального определения кластеров
'''
1 - elbow
2 - silhoulette
'''

# метод кластеризации

'''
1 - k-means
2 - AgglomerativeClustering
3 - SpectralClustering

'''

In [ ]:
condition

In [ ]:
condition.to_excel('condition_results.xlsx',encoding='utf-8')